In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pickle import *
import folium
from folium.plugins import HeatMap
import branca

In [2]:
f = open("dati_simulazione_prova", "rb")
tt = load(f)
l_temp = len(tt)
U = []
for i in range(l_temp-1):
    U.append(load(f))

In [3]:
I = np.zeros(32) # Strade del tipo I: strade principali interne alla rete
for i in range(len(I)+1):
    if i in [1,2,12,13,14,15,27,28,29,30]:
        I[i-1] = 170
    elif i in [5,6]:
        I[i-1] = 450
    elif i in [9,11,16,19,22]:
        I[i-1] = 150
    elif i in [17,18,20,21,23,24,25,26]:
        I[i-1] = 85
    elif i in [10,31,32]:
        I[i-1] = 46
I[2] = 140
I[3] = 110
I[6] = 56
I[7] = 16

j = np.ones(10)*8 # Strade del tipo j: strade interne agli incroci

r = np.zeros(8) # Strade del tipo r: strade interne alle rotonde
for i  in range(len(r)+1):
    if i in [7,8]:
        r[i-1] = 4
    elif i in [3,6]:
        r[i-1] = 12
r[0] = 8
r[1] = 10
r[3] = 17
r[4] = 14

In [4]:
dx =[I[i]/100 for i in range(len(I))] # Per le strade interne prendiamo uno step del 1%
dx.append(np.float64(0.8)) # Per le strade degli incroci prendiamo un step del 10%
dx.extend(r[k]/10 for k in range(len(r))) # Per le strade delle rotonde prendiamo uno step del 10%
dx_min = min(dx)

xx = [np.arange(dx[i]/2, I[i], dx[i]) for i in range(len(I))]
xx.extend(np.arange(dx[i]/2, j[i], dx[i]) for i in range(len(j)))
xx.extend(np.arange(dx[i]/2, r[i], dx[i]) for i in range(len(r)))

In [5]:
def retta(punti: list, xx: np.array)-> np.array:
    return ((xx-punti[0][0])/(punti[1][0]-punti[0][0])*(punti[1][1]-punti[0][1]))+punti[0][1]

In [6]:
loc = []
loc.append([[45.51359, 9.21432],[45.51232 , 9.21327]])
loc.append([[45.51229, 9.21336] ,[45.51354 , 9.21442]])
loc.append([[45.51408, 9.21272] , [45.51364 , 9.21427] ,[45.51359 , 9.21432] ])
loc.append([[45.51232, 9.21327] , [45.51270 , 9.21194]])
loc.append([[45.51232, 9.21327]  , [45.51001 , 9.21198]  , [45.50987 , 9.21191]  , [45.50976 , 9.21180]  , [45.50969 , 9.21165]  , [45.50932 , 9.21023] ])
loc.append([[45.50922, 9.21025]  , [45.50934 , 9.21054]  , [45.50964 , 9.21175]  , [45.50974 , 9.21191]  , [45.50985 , 9.21200]  , [45.51229 , 9.21336] ])
loc.append([[45.50938, 9.21014]  , [45.50948 , 9.21012]  , [45.50987 , 9.21033] ])
loc.append([[45.50936 , 9.20998],[45.50953, 9.21003]])
loc.append([[45.51002, 9.20820]  , [45.50953 , 9.21003]])
loc.append([[45.50989, 9.21024], [45.50953 , 9.21003]])
loc.append([[45.50989, 9.21024]  , [45.51041 , 9.20840] ])
loc.append([[ 45.51135, 9.21106]  ,[ 45.50989 , 9.21024] ])
loc.append([ [45.50987, 9.21033 ] , [45.51132 , 9.21116] ])
loc.append([[45.51132, 9.21116]  , [45.51270 , 9.21194] ])
loc.append([ [45.51273, 9.21184 ] , [45.51135 , 9.21106] ])
loc.append([ [45.51186, 9.20923]  , [45.51135 , 9.21106] ])
loc.append([[45.51270, 9.21194]  , [45.51339 , 9.21233]] )
loc.append([[45.51341, 9.21223]  , [45.51273 , 9.21184]] )
loc.append([[45.51273 , 9.21184]  , [45.51324,9.21001]] ) # I19
loc.append([[45.51339, 9.21233]  , [45.51408 , 9.21272] ]) # I20
loc.append( [[45.51410, 9.21263]  , [45.51341 , 9.21223] ]) # I21
loc.append([[ 45.51462, 9.21079]  , [45.51410 , 9.21263]  ]) #I22
loc.append([[45.51464, 9.21071 ] , [45.51396 , 9.21030] ]) # I23
loc.append( [[45.51392, 9.21040]  , [ 45.51462, 9.21079] ]) # I24
loc.append([[45.51396 , 9.21030] , [45.51327 , 9.20992] ])
loc.append([[45.51324,9.21001]  , [45.51392, 9.21040] ]) # I26
loc.append([[45.51327, 9.20992]  , [45.51189 , 9.20914] ])
loc.append([ [45.51186, 9.20923] ,[45.51324,9.21001] ])
loc.append([[45.51189, 9.20914] , [45.51043 , 9.20833] ])
loc.append([[45.51041, 9.20840] , [45.51186 , 9.20923] ])
loc.append([[ 45.51043, 9.20833 ], [45.51005 , 9.20811] ])
loc.append( [[45.51002, 9.20820], [45.51041 , 9.20840 ]])


In [7]:
yy = []
zz = []
for i in range(len(I)):
    tmp2 = np.zeros(len(xx[i]))
    for k in range(len(loc[i])-1):
        if i not in [2,4,5,6]:
            tmp2 = np.linspace(loc[i][k][0],loc[i][k+1][0], len(xx[i]))
    zz.append(tmp2)

lat = zz[0]
for i in range(1,len(I)):
    lat = np.concatenate((lat,zz[i]))

for i in range(len(I)):
    tmp = np.zeros(len(zz[i]))
    for k in range(len(loc[i])-1):
        if i not in [2,4,5,6]:
            tmp = retta([loc[i][k], loc[i][k+1]],zz[i])
    yy.append(tmp)

lon = yy[0]
for i in range(1,len(I)):
    lon = np.concatenate((lon,yy[i]))

In [8]:
list_colors = [
    "#00FF00",
    "#12FF00",
    "#24FF00",
    "#35FF00",
    "#47FF00",
    "#58FF00",
    "#6AFF00",
    "#7CFF00",
    "#8DFF00",
    "#9FFF00",
    "#B0FF00",
    "#C2FF00",
    "#D4FF00",
    "#E5FF00",
    "#F7FF00",
    "#FFF600",
    "#FFE400",
    "#FFD300",
    "#FFC100",
    "#FFAF00",
    "#FF9E00",
    "#FF8C00",
    "#FF7B00",
    "#FF6900",
    "#FF5700",
    "#FF4600",
    "#FF3400",
    "#FF2300",
    "#FF1100",
    "#FF0000",
]

In [9]:
my_color_map = branca.colormap.LinearColormap(colors= list_colors)

In [10]:
import folium.map
feat1 = folium.map.FeatureGroup(name = 't = {:.2f}s'.format(tt[0]))

m = folium.Map(location = [45.51200,9.21147], zoom_start = 17, tiles = "CartoDB Positron", control_scale= True)
for i in range(len(I)):
    colorline = folium.ColorLine(positions = list(zip(zz[i],yy[i])), colors = U[0][i], colormap = my_color_map, weight = 5)
    feat1.add_child(colorline)
m
feat1.add_to(m)

feat2 = folium.map.FeatureGroup(name = 't = {:.2f}s'.format(tt[-1]))
for i in range(len(I)):
    colorline = folium.ColorLine(positions = list(zip(zz[i],yy[i])), colors = U[-1][i], colormap = my_color_map, weight = 5)
    feat2.add_child(colorline)
m
feat2.add_to(m)
m.add_child(my_color_map)
folium.map.LayerControl(collapsed= False).add_to(m)
m